# Classifying Fashion MNIST Dataset using Convolutional Neural Network
### Author: Samuel Adamson
### Tensorflow, Tensorflow Datasets, Numpy, MatPlotLib
### Last Edited 01/01/2022

In [5]:
# Imports
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os
import math

# Set logging for errors only
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Get Fashion MNIST data from Tensorflow Datasets.
<br>
Training Data: 60,000 Entries
<br>
Testing Data: 10,000 Entries

In [ ]:
# Get training / testing datasets ( Fashion MNIST )
# Return Type: Tuple
# @RETURN: ( [ training data, testing data ], class names, num_training, num_testing )
def get_data():
    # Disable progress bar on datasets
    tfds.disable_progress_bar()

    # Get data / metadata, separate out training and testing data
    dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
    training, testing = dataset['train'], dataset['test']

    # Parse out class names
    class_names = metadata.features['label'].names
    
    # Data for training
    num_training = metadata.splits['train'].num_examples # Number of training examples
    num_testing = metadata.splits['test'].num_examples  # Number of testing examples

    return ([training, testing], class_names, num_training, num_testing)

# Preprocess data for training
# Return Type: List
# @RETURN: [ training data, testing data, class names ]
def preprocess(data):

    # Normalize data
    # Scale data from 0 - 255 to 0 - 1
    def normalize(images, labels):
        # Divide by 255
        images = tf.cast(images, tf.float32) / 255
        return images, labels

    # Normalize Training and Testing Data
    data[0] = data[0].map(normalize)
    data[1] = data[1].map(normalize)

    return data


# Curate Data
data, class_names, num_training, num_testing = get_data()
data = preprocess(data)

In [ ]:
# Verify Data -- Reshape and show images
# Return Type: None
# @RETURN: None
def example_data(sub_data):
    # Format plot figure size
    plt.figure(figsize=(12,12))

    # Iterate through subsidiary data
    for i, (image, label) in enumerate(sub_data):
        # Reshape Image
        image = image.numpy().reshape((28,28))
        # Plot
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image, cmap=plt.cm.binary)
        plt.xlabel(class_names[label])

    # Save plot
    !mkdir figs
    plt.savefig('./figs/example_data.png')
    # Show Plot
    plt.show()

# Show example data 
example_data(data[1].take(20))

Create model, Train Model, Evaluate accuracy of model after training
Model contains three layers:

*Input Layer* - Convolutional 2d, 3x3 filter, 32 output images
<br>
*Hidden Layers* <br>
&emsp; Max Pooling 2x2 window size, stride size 2 <br>
&emsp; Convolutional 2d, 3x3 filter, 64 output images <br>
&emsp; Max Pooling 2x2 window size, stride size 2 <br>
&emsp; Densely connect layer of 128 nodes <br>
*Output Layer* - 10 Node softmax layer, each node represents one class of clothing

Note: If using Google Colab -- enable GPU to speed up testing

In [ ]:
# Create and Train Model:
#   Input Layer: Convolutional 2d 3x3 filter, 32 output images
#   Hidden Layers:
#       Max Pooling 2x2 window size, stride 2
#       Convolutional 2d 3x3 filter, 64 output images
#       Max Pooling 2x2 window size, stride 2
#       Flatten layer -- convert to 1d array
#       Densely connect layer 128 nodes
#   Output Layer: Softmax 10 nodes 10 classes, sum of all 10 nodes = 1
# Return Type: Keras Model
# @RETURN: ( Model )
def ct_model(dataset, num_training):
    # Define model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2,2), strides=2),
        tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    # Configure Model for training
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    # Training Settings, cache data, shuffle data, sort into batch sizes
    batch_size = 32
    dataset = dataset.cache().repeat().shuffle(num_training).batch(batch_size)
    # Conduct training
    #   Max epochs: 5, steps per epoch: number of batches
    model.fit(dataset, epochs=8, steps_per_epoch=math.ceil(num_training/batch_size))

    # Save model upon completion
    # !mkdir model
    # model.save('./model/fashion_mnist_classification')

    return model


# Evaluate accuracy
# Return Type: None
# @RETURN: None
def eval_accuracy(model, dataset, num_testing):
    # Configure data
    batch_size = 32
    dataset = dataset.cache().batch(batch_size)

    # Evaluate
    loss, accuracy = model.evaluate(dataset, steps=math.ceil(num_testing/batch_size))
    print('Accuracy: ', accuracy)


# Create and train model
model = ct_model(data[0], num_training)
# Test Accuracy using testing data
eval_accuracy(model, data[1], num_testing)

Predictions -- Make a prediction based on testing images

In [ ]:
# Plot an image with green label == correct prediction, red label == incorrect prediction
# Return Type: None
# @RETURN: None
def plot_image(prediction, actual_label, image, class_names, index=0):
    # Store prediction, correct label, and corresponding image at index
    prediction, image = prediction[index], image[index]

    # Configure plot
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    # Display data as image
    plt.imshow(image[...,0], cmap=plt.cm.binary)

    # Predicted class, highest probability
    predicted_label = np.argmax(prediction)
    # Store color green = correct, red = incorrect
    color = 'red'
    if predicted_label == actual_label:
        color = 'green'

    # Plot labels
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                         100*np.max(prediction),
                                         class_names[actual_label]),
                                         color=color)


# Get image from test dataset
temp_dataset = data[1].take(1)
# Extract image
for image, label in temp_dataset:
    image = image.numpy()
    label = label.numpy()


# Convert for passing to NN
image = np.array([image])
# Make prediction
prediction = model.predict(image)

# Plot image with prediction
plot_image(prediction, label, image, class_names)